In [ ]:
# failed attempt, not working!!

In [1]:
with open("debug.txt", "r") as f:
    lines = f.readlines()

pattern = []
for line in lines:
    pattern.append([int(char) for char in line.strip()])
# make pattern a tuple
pattern = tuple(tuple(row) for row in pattern)

In [2]:
pattern

((2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3),
 (3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3),
 (3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4),
 (3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2),
 (4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6),
 (1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4),
 (4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6),
 (3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3),
 (4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7),
 (4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3),
 (1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3),
 (2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5),
 (4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3))

In [3]:
def generate_next_pos(cur_row, cur_col, last_direction, last_direction_count, pattern_row_len, pattern_col_len):
    # can move up/down/left/right
    # if has been on the same direction for more than 3 steps, need to change direction

    possible_next_steps = [
        # up
        [cur_row - 1, cur_col, "up", 0],
        # down
        [cur_row + 1, cur_col, "down", 0],
        # left
        [cur_row, cur_col - 1, "left", 0],
        # right
        [cur_row, cur_col + 1, "right", 0],
    ]

    # can not go back
    if last_direction == "up":
        possible_next_steps.remove([cur_row + 1, cur_col, "down", 0])
    elif last_direction == "down":
        possible_next_steps.remove([cur_row - 1, cur_col, "up", 0])
    elif last_direction == "left":
        possible_next_steps.remove([cur_row, cur_col + 1, "right", 0])
    elif last_direction == "right":
        possible_next_steps.remove([cur_row, cur_col - 1, "left", 0])

    # remove out of border ones
    possible_next_steps = [step for step in possible_next_steps if step[0] >= 0 and step[0] < pattern_row_len and step[1] >= 0 and step[1] < pattern_col_len]

    # fix the direction count
    for step in possible_next_steps:
        if step[2] == last_direction:
            step[3] = last_direction_count + 1
    
    # remove if same direction more than 3
    # TODO: selection point?
    possible_next_steps = [step for step in possible_next_steps if step[3] < 3]

    

    return possible_next_steps

In [4]:
pattern_row_len, pattern_col_len = len(pattern), len(pattern[0])
cur_row, cur_col = 0, 0

In [5]:
from functools import cache

In [6]:
def estimate_distance(cur_row, cur_col, target_row, target_col):
    # manhattan distance
    return abs(cur_row - target_row) + abs(cur_col - target_col)

In [7]:
def get_middle_weights(path_str, pattern):
    path_points = [i for i in path_str.split("|") if i != ""]
    path_points = [[int(i) for i in point.split(",")] for point in path_points]
    weights = []
    for point in path_points:
        weights.append(pattern[point[0]][point[1]])
    return weights

In [8]:
def find_route_to_end_a_star(cur_row, cur_col, last_direction, last_direction_count, pattern):

    # target is the bottom right corner
    explored_list = []
    not_explored_list = []

    pattern_row_len, pattern_col_len = len(pattern), len(pattern[0])
    target_row, target_col = pattern_row_len - 1, pattern_col_len - 1

    # step structure
    # cur_row, cur_col, last_direction, last_direction_count, f=g+h, parent_cords 

    # add the first step
    not_explored_list.append([cur_row, cur_col, last_direction, last_direction_count, 0 + estimate_distance(cur_row, cur_col, target_row, target_col), ""])

    while len(not_explored_list) > 0:
        # find the one with the smallest f
        min_f = 99999999
        min_f_index = 0
        for i, step in enumerate(not_explored_list):
            f = step[4]
            if f < min_f:
                min_f = f
                min_f_index = i
        
        # debug
        print("not explored", not_explored_list)

        # pop the one with the smallest f
        cur_step = not_explored_list.pop(min_f_index)
        print(f"min_f: {min_f}, min_f_index: {min_f_index}", "cur_step:", cur_step)

        # check if it is the target
        if cur_step[0] == pattern_row_len - 1 and cur_step[1] == pattern_col_len - 1:
            print("at target, path=", cur_step[5])
            middle_weights = get_middle_weights(cur_step[5], pattern)
            return sum(middle_weights) + pattern[-1][-1]
        
        # add to explored list
        explored_list.append(cur_step)

        # generate next steps
        possible_next_steps = generate_next_pos(cur_step[0], cur_step[1], cur_step[2], cur_step[3], pattern_row_len, pattern_col_len)

        # add distance estimate on them
        current_weight = pattern[cur_step[0]][cur_step[1]]
        for step in possible_next_steps:
            # TODO: ???
            previous_weights = get_middle_weights(cur_step[5], pattern)
            step.append(sum(previous_weights) + current_weight + estimate_distance(step[0], step[1], target_row, target_col))
            step.append(cur_step[5] + str(cur_step[0]) + "," + str(cur_step[1]) + "|")

        # add to not explored list
        for step in possible_next_steps:
            # check if it is in the explored list
            if step in explored_list:
                continue
            # check if it is in the not explored list
            if step in not_explored_list:
                # check if the f is smaller
                for i, not_explored_step in enumerate(not_explored_list):
                    if step == not_explored_step:
                        if step[4] < not_explored_step[4]:
                            # replace
                            not_explored_list[i] = step
                        break
            else:
                # add to not explored list
                not_explored_list.append(step)


In [9]:
# find_route_to_end_a_star(0,1, "right", 1, pattern)